This program processes student essays that outline their approach to solving a problem. We use the BERT model from the simple transformers library to make predictions. BERT transforms raw text into contextualized word embeddings. The target variable is correctnesss on the problem.  

In [ ]:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import textblob
from textblob import TextBlob

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


# Import data from google drive
downloaded = drive.CreateFile({'id':"1BZobq1AdzZNT8Z6h9PqR3EG8hy0LfyTm"})   # replace the id with id of file you want to access
downloaded.GetContentFile('172Q3P6.xls')   

#https://drive.google.com/file/d/1yfOgSHc3QdQixetyQFr2GGbGIPLBmX6Q/view?usp=sharing
downloaded = drive.CreateFile({'id':"1yfOgSHc3QdQixetyQFr2GGbGIPLBmX6Q"})   # replace the id with id of file you want to access
downloaded.GetContentFile('P172-Sp21-Quiz3_P6-LastAttempt_172ALL-Data.xlsx')  

train = pd.read_excel('172Q3P6.xls')
test = pd.read_excel('P172-Sp21-Quiz3_P6-LastAttempt_172ALL-Data.xlsx')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Load previously processed data (.pkl)

In [4]:
import gc
import pickle
from google.colab import drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

trainPKLread = open('drive/My Drive/trainNLP.pkl','rb')
train = pickle.load(trainPKLread)
trainPKLread.close()

testPKLread = open('drive/My Drive/testNLP.pkl','rb')
test = pickle.load(testPKLread)
testPKLread.close()

Mounted at /content/drive


Function and class to process data

In [ ]:
#Function clean takes a string (essay) as an argument.
#       -Removes Stop Words
#       -Removes Unnecessary chars
#       -Contextually unaware spelling correction

def clean(txt):
    # Set stopwords (english from NLTK library)
    stop = stopwords.words('english')
    # Remove punctuation and symbols (equations) which are essentially noise
    punc = ['0','1','2','3','4','5','6','7','8','9','!','@','#','$','%','^','&','*','(',')','~','`','[',']','{','}','|','\'','?','>','<',':',';',',','+','_','-','=']
    # Remove new line characters and split on space character
    txt = txt.replace('\n','').split(' ')
    # 
    txt = [word for word in txt if word not in stop]
    text = []
    for word in txt:
      chars = [char for char in word if char not in punc]
      word = ''.join(chars)
      if word != '':
        text.append(word.lower())
    text = ' '.join(text)
    text = TextBlob(text)
    text = str(text.correct())
    return text

class dataPrep:
  """
  --------------------------dataPrep--------------------------------
  Class has two methods that are responsible for structuring data 
  in a format that makes BERT happy. Namely, a dx2 dimensional array 
  or dataframe with one column marked 'text' and another marked 'label'

  ----Attributes----
  d1 ==> pandas dataframe with test data
  d2 ==> pandas dataframe with training data

  ----Methods----
  set1Format ==> drops nuissance rows and renames columns. Applies clean function to
                 each essay

  set2Format ==> drops nuissance rows and renames columns. Applies clean function to 
                  each essay

  """
  
  def __init__(self,testdf,traindf):
    self.d1 = testdf
    self.d2 = traindf
  
  def set1Format(self):
    try:
      self.d1.drop([0,1,2,3,4],inplace=True)
    except:
      pass
    self.d1[['PUID','label','text']] = self.d1[['Unnamed: 2','Unnamed: 8','Unnamed: 34']]
    self.d1 = self.d1.loc[:,['PUID','label','text']]
    self.d1.dropna(how='any',inplace=True)
    self.d1.drop_duplicates(subset='PUID',inplace=True)
    self.d1 = self.d1.loc[:,['text','label']]
    self.d1['text'] = self.d1['text'].apply(lambda x: clean(x))
    return self.d1
  
  def set2Format(self):
    self.d2[['User','label','text']] = self.d2[['Username','Awarded (OA)','A5R']]
    self.d2 = self.d2.loc[:,['User','label','text']]
    self.d2.dropna(how='any',inplace=True)
    self.d2.drop_duplicates(subset='User',inplace=True)
    self.d2 = self.d2.loc[:,['text','label']]
    self.d2['text'] = self.d2['text'].apply(lambda x: clean(x))
    return self.d2

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
# dataSet = dataPrep(test,train)

In [ ]:
# train = dataSet.set2Format()
# test = dataSet.set1Format()

Pickles the processed data sets

In [ ]:
# create and store train and test .pkl files 

# import gc
# import pickle
# from google.colab import drive

# trainNLPout = open('drive/My Drive/trainNLP.pkl','wb')
# pickle.dump(train, trainNLPout)
# trainNLPout.close()


# testNLPout = open('drive/My Drive/testNLP.pkl','wb')
# pickle.dump(test, testNLPout)
# testNLPout.close()


In [ ]:
!pip install simpletransformers

In [6]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import wandb

In [35]:
args= ClassificationArgs()
args.overwrite_output_dir = True
args.num_train_epochs = 4
args.reprocess_input_data = True
args.use_multiprocessing = True
args.evaluate_during_training = False
args.use_early_stopping = True
args.train_batch_size= 50
args.polynomial_decay_schedule_lr_end = 1e-6
'''
args.wandb_project = "172 classification"

sweep_config = {
    "name": "172 classification",
    "method": "Bayes",
    "metric": {"name": "train_loss", "goal": "minimize"},
    "parameters": {
        "num_train_epochs": {"min": 1, "max": 40},
        "learning_rate": {"min": 0, "max": 4e-4},
    },
    "early_terminate": {"type": "hyperband", "min_iter": 6,},
}

sweep_id = wandb.sweep(sweep_config, project="172 classification")
'''

# Create a TransformerModel with modified attributes
model = ClassificationModel("bert", "bert-base-cased",num_labels=2,use_cuda=True,args=args)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Model validation using WandB

In [32]:
train,val = train_test_split(train,test_size=0.3)

In [33]:
# model.train_model(train)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/493 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/10 [00:00<?, ?it/s]

(40, 0.548869339376688)

In [34]:
# model.evaluate(val,output_dir='output')

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/212 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/27 [00:00<?, ?it/s]

({'auprc': 0.8294727304447622,
  'auroc': 0.8468424334194352,
  'eval_loss': 0.4921654595269097,
  'fn': 20,
  'fp': 25,
  'mcc': 0.5763391210994668,
  'tn': 84,
  'tp': 83},
 array([[ 1.2890625 , -0.87646484],
        [ 1.2734375 , -0.828125  ],
        [-0.82861328,  0.83105469],
        [ 1.30957031, -0.87304688],
        [-0.72753906,  0.7890625 ],
        [ 1.16992188, -0.71435547],
        [-0.94677734,  0.70800781],
        [ 1.30371094, -0.85498047],
        [ 0.73876953, -0.28833008],
        [ 1.29589844, -0.82617188],
        [ 1.13476562, -0.86035156],
        [ 1.22949219, -0.88232422],
        [-0.44677734,  0.65332031],
        [-0.68603516,  0.77148438],
        [ 0.60986328, -0.22619629],
        [-0.77099609,  0.80957031],
        [-0.74414062,  0.75244141],
        [ 1.23925781, -0.78173828],
        [-0.96582031,  0.76855469],
        [ 1.25292969, -0.75048828],
        [-0.27587891,  0.54541016],
        [-0.51171875,  0.73535156],
        [-0.74902344,  0.80322266

In [36]:
model.train_model(train,eval_df=val)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/493 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/10 [00:00<?, ?it/s]

(40, 0.6069374553859234)

In [37]:
model.evaluate(test,output_dir='output')

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/1480 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/185 [00:00<?, ?it/s]

({'auprc': 0.7808251355210292,
  'auroc': 0.81578807050833,
  'eval_loss': 0.5342963966163429,
  'fn': 283,
  'fp': 93,
  'mcc': 0.49485172416164264,
  'tn': 708,
  'tp': 396},
 array([[-0.27172852,  0.74658203],
        [ 0.21191406, -0.11340332],
        [-0.0859375 ,  0.62255859],
        ...,
        [ 0.52880859, -0.89306641],
        [ 0.66845703, -1.03808594],
        [-0.4206543 ,  1.03613281]]),
 [[1,
   1,
   1,
   1,
   0,
   1,
   0,
   1,
   1,
   0,
   0,
   0,
   1,
   0,
   1,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   1,
   1,
   0,
   0,
   1,
   1,
   1,
   1,
   0,
   0,
   1,
   1,
   0,
   1,
   1,
   1,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   1,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   1,
   0,
   1,
   0,
   1,
   0,
   0,
   0,
   1,
   0,
   1,
   1,
   1,
   0,
   0,
   0,
   1,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0

In [38]:
preds = model.predict(test['text'].to_list())

  0%|          | 0/1480 [00:00<?, ?it/s]

  0%|          | 0/185 [00:00<?, ?it/s]

In [17]:
from sklearn.metrics import classification_report, confusion_matrix

In [39]:
print(classification_report(np.array(test['label'],dtype=int),preds[0]))

              precision    recall  f1-score   support

           0       0.71      0.88      0.79       801
           1       0.81      0.58      0.68       679

    accuracy                           0.75      1480
   macro avg       0.76      0.73      0.73      1480
weighted avg       0.76      0.75      0.74      1480



In [40]:
print(confusion_matrix(np.array(test['label'],dtype=int),preds[0]))

[[708  93]
 [283 396]]


In [22]:
np.array(test['label'])

array([1, 1, 1, ..., 0, 0, 1], dtype=object)